In [1]:
import pandas as pd
import numpy as np
import plotly.express as px


#PART II : Facebook's Prophet (Greykite)

##Import

Start by installing greykite library in Colab

In [ ]:

!pip install greykite -q



In [3]:
import datetime
import pandas as pd
import plotly.express as px
from greykite.framework.templates.autogen.forecast_config import ForecastConfig, MetadataParam, ModelComponentsParam
from greykite.framework.benchmark.data_loader_ts import DataLoaderTS
from greykite.framework.templates.forecaster import Forecaster

## Import the data

In [4]:
df_preproc = pd.read_csv("/content/PA75016_prepro.csv")
df_raw = pd.read_csv("/content/PA75016_prepro (1).csv")

In [5]:
df_preproc

,Date_time,PM25,PM10,NO2,Pollution_peak,ATMO,sin_Month,cos_Month,sin_day,cos_day,confinement
0,2018-01-01,-0.001422,-0.048074,-0.840686,0.0,0,0.000000,1.0,-0.781831,0.623490,0
1,2018-01-02,-0.001422,-0.048074,-0.840686,0.0,0,0.000000,1.0,0.000000,1.000000,0
2,2018-01-03,-0.001422,-0.048074,-0.840686,0.0,0,0.000000,1.0,0.781831,0.623490,0
3,2018-01-04,-0.001422,-0.048074,-0.840686,0.0,0,0.000000,1.0,0.974928,-0.222521,0
4,2018-01-05,-0.025605,-0.131895,-0.840686,0.0,0,0.000000,1.0,0.433884,-0.900969,0
...,...,...,...,...,...,...,...,...,...,...,...
1770,2022-11-09,-0.504979,-0.603390,-0.507353,0.0,0,-0.866025,0.5,0.781831,0.623490,0
1771,2022-11-10,0.012802,-0.389522,-0.465686,0.0,0,-0.866025,0.5,0.974928,-0.222521,0
1772,2022-11-11,-0.315789,-0.951002,-1.296569,0.0,0,-0.866025,0.5,0.433884,-0.900969,0
1773,2022-11-12,1.544808,0.395069,-0.205882,0.0,1,-0.866025,0.5,-0.433884,-0.900969,0


In [6]:
df_raw

,Date_time,PM25,PM10,NO2,Pollution_peak,ATMO,sin_Month,cos_Month,sin_day,cos_day,confinement
0,2018-01-01,13.00,29.90,71.1,0,0,0.000000,1.0,-0.781831,0.623490,0
1,2018-01-02,13.00,29.90,71.1,0,0,0.000000,1.0,0.000000,1.000000,0
2,2018-01-03,13.00,29.90,71.1,0,0,0.000000,1.0,0.781831,0.623490,0
3,2018-01-04,13.00,29.90,71.1,0,0,0.000000,1.0,0.974928,-0.222521,0
4,2018-01-05,12.83,28.54,71.1,0,0,0.000000,1.0,0.433884,-0.900969,0
...,...,...,...,...,...,...,...,...,...,...,...
1770,2022-11-09,9.46,20.89,84.7,0,0,-0.866025,0.5,0.781831,0.623490,0
1771,2022-11-10,13.10,24.36,86.4,0,0,-0.866025,0.5,0.974928,-0.222521,0
1772,2022-11-11,10.79,15.25,52.5,0,0,-0.866025,0.5,0.433884,-0.900969,0
1773,2022-11-12,23.87,37.09,97.0,0,1,-0.866025,0.5,-0.433884,-0.900969,0


## Make Forecast


### Train the model

[`Check the documentation of greykite`](https://github.com/linkedin/greykite)

We want to use the PROPHET model from Facebook to train your model and make a prediction for future turnover

Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data.

Set the model_template as 'PROPHET' to use the model from facebook.  
 Forecast your model for a horizon of 1 month (in order to predict your last month of december)



Define the `ForecastConfig` with :
- `metadata_param` : tell the model what is your time and value columns in your dataset

  -> define `time_col`, `value_col`

  -> define `train_end_date` as datetime.datetime to tell the model when to stop the training.
- `model_template` : select 'PROPHET' since is the model we want to use
- `forecast_horizon` : how many steps ahead we want to predict
- `coverage` : 95% of actuals should fall within the prediction intervals (`coverage=0.95`)

We want to use the same trainning values than our first ARIMA model (values until 1st Dec 2021) and test our prediction with the same values (month of December 2021). 

This model provides nice vizualisations if we give it all the `df` dataset and fix the training values with `train_end_date` variable. 

Also, this model will use cross-validation methods to fit and test inside the training dataset to find the best parameters.

In [65]:
# $CHA_BEGIN
config = ForecastConfig(
     metadata_param= MetadataParam(
      time_col = "Date_time",
      value_col = "ATMO",
      train_end_date = datetime.datetime(2022, 11, 1)),
     model_template="PROPHET",
     forecast_horizon=7,
     coverage=0.95 )
# $CHA_END

In [66]:
import logging
forecaster = Forecaster()
logging.getLogger('prophet').setLevel(logging.WARNING) #not printing log
logging.getLogger('cmdstanpy').setLevel(logging.WARNING) #not printing log
result = forecaster.run_forecast_config(df=df_raw, config=config) 
model_prophet = result.model

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:418: UserWarning:

y_true contains 0. MAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/sklearn/sklearn_scorer.py:174: UserWarning:

Score is undefined for this split, setting to `np.nan`.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:435: UserWarning:

y_true contains 0. MedAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:418: UserWarning:

y_true contains 0. MAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/sklearn/sklearn_scorer.py:174: UserWarning:

Score is undefined for this split, setting to `np.nan`.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:435: UserWarning:

y_true contains 0. MedAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:153: UserWarning:

3 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.8/di

Now that your model predicted a forecast, you will be able to access these results

### Evaluate your forecast

We can access the result of the prediction with `result.forecast` and we can evaluate the performance with `result.forecast.test_evalution`. 
Find the same metric as the Arima model and assign it to a `score_prophet` variable.

In [67]:
df_result = result.forecast.df
df_result

,Date_time,actual,forecast,forecast_lower,forecast_upper
0,2018-01-01,0.0,1.526372,0.244095,2.844578
1,2018-01-02,0.0,0.754153,-0.457703,2.104134
2,2018-01-03,0.0,1.503358,0.226932,2.741368
3,2018-01-04,0.0,1.316217,-0.006079,2.565143
4,2018-01-05,0.0,1.553991,0.344132,2.755142
...,...,...,...,...,...
1768,2022-11-04,0.0,1.586526,0.288674,2.909284
1769,2022-11-05,0.0,0.405257,-0.864182,1.695904
1770,2022-11-06,0.0,0.301633,-1.043347,1.572376
1771,2022-11-07,0.0,0.401660,-0.858939,1.674125


In [68]:
df_result['forecastclass']=round(df_result['forecast']).astype(int)

In [69]:
df_result.dropna(inplace=True)

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(df_result['actual'], df_result['forecastclass'])

0.5276836158192091

In [33]:
print(f"The MAPE of Prophet model is {round(result.forecast.test_evaluation['MAPE'])}%")

The MAPE of Prophet model is 62%


Is this model better than ARIMA ?

We also have access to `result.backtest` information from the cross-validation method but we're not going to use this today. 

### Plot the forecast

Greykite library has its own fancy plot function that integrate the upper and lower bound of the prediction.

Remember that you can zoom on plotly graph

In [71]:
result.forecast.plot()

### Custom the model with specific holidays and seasonality

Facebook Prophet model proposes us to integrate holidays and seasonality to the algorithm.

We would like to custom them based on our understanding of the data.

Try to query the table `sales` in BigQuery to find out where the higher number of sales is made. 
We will specify the model which country holidays should impact most the turnover.

Do not erase your `df` dataset, choose an other name !

<details>
    <summary><i>Hint</i></summary>
use `COUNT(*)`, `GROUP BY` and `ORDER BY` to query the top 3 countries which have the higher number of sales. 
</details>        

In [72]:
# $CHA_BEGIN
config_holidays = ForecastConfig(
    metadata_param = MetadataParam(
      time_col = "Date_time",
      value_col = "ATMO",
      train_end_date = datetime.datetime(2022, 11, 1)),
    model_template = "PROPHET",
    forecast_horizon = 7,
    coverage = 0.95,
    model_components_param = ModelComponentsParam(
     seasonality={"yearly_seasonality": True, 'daily_seasonality':True},
     events = {
         "holiday_lookup_countries": ["France"],
         "holiday_pre_num_days": [0],
         "holiday_post_num_days": [0],
     }))


# $CHA_END

In [73]:
result_custom = Forecaster().run_forecast_config(
    df = df_preproc,
    config = config_holidays)
model_prophet_custom = result_custom.model

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:418: UserWarning:

y_true contains 0. MAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/sklearn/sklearn_scorer.py:174: UserWarning:

Score is undefined for this split, setting to `np.nan`.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:435: UserWarning:

y_true contains 0. MedAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:418: UserWarning:

y_true contains 0. MAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/sklearn/sklearn_scorer.py:174: UserWarning:

Score is undefined for this split, setting to `np.nan`.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:435: UserWarning:

y_true contains 0. MedAPE is undefined.

/usr/local/lib/python3.8/dist-packages/greykite/common/evaluation.py:153: UserWarning:

3 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.8/di

Assign the MAPE to the variable `score_prophet_custom`

In [74]:
df_result_custom = result_custom.forecast.df
df_result_custom

,Date_time,actual,forecast,forecast_lower,forecast_upper
0,2018-01-01,0.0,1.354138,-0.100533,2.651545
1,2018-01-02,0.0,1.523291,0.157681,2.997038
2,2018-01-03,0.0,1.624789,0.207891,3.038805
3,2018-01-04,0.0,1.571230,0.154775,2.918484
4,2018-01-05,0.0,1.669557,0.295783,3.107232
...,...,...,...,...,...
1768,2022-11-04,0.0,0.605199,-0.770385,1.891977
1769,2022-11-05,0.0,0.425532,-0.926196,1.774357
1770,2022-11-06,0.0,0.349553,-1.067446,1.719128
1771,2022-11-07,0.0,0.458632,-0.922661,1.881002


In [75]:
df_result_custom['forecastclass']=round(df_result_custom['forecast']).astype(int)

In [76]:
df_result_custom.dropna(inplace=True)

In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(df_result_custom['actual'], df_result_custom['forecastclass'])

0.4497175141242938

In [ ]:
print(f"The MAPE of Prophet model is {round(result_custom.forecast.test_evaluation['MAPE'])}%")

In [79]:
result_custom.forecast.plot_components()

ImportError: ignored

<Figure size 648x1080 with 5 Axes>

The time serie will be the addition of all these series.